In [7]:
import pandas as pd
import numpy as np
from mca import diagsvd
# import mca
import functools
import sklearn.model_selection as ms
from sklearn.preprocessing import StandardScaler as stdc
import gc
import _pickle as pickle
categorical = ['source_system_tab','source_type','gender','city']
numerical = ['song_length', 'song_year','bd']

def standarize_data(df):
	df = stdc().fit_transform(df)
	return df



In [8]:
def split(X, y, proportion):
	print('Train \nRaw data: \n ', 'Shape: ', X.shape, 'Type: ', type(X))
	print('Target \nRaw data: \n ', 'Shape: ', y.shape)
	# I split the data between the X and the target value

	# X is a dataframe not a np.array
	print('Size of X: ', X.shape, '\nSize of y: ', y.shape)
	(X_train, X_test, y_train, y_test) = ms.train_test_split(X, y, test_size=proportion, random_state=1, stratify=y)
	(X_test, X_val, y_test, y_val) = ms.train_test_split(X_test, y_test, test_size=.5, random_state=1, stratify=y_test)
	print('\n New train shape: ', X_train.shape, ' \n New test shape: ', X_test.shape, '\n New val shape: ',
	      X_val.shape, '\n Type: ',type(X_train))

	return X_train, X_test, X_val, y_train, y_test, y_val


# Partitioning

In [9]:
file = 'def_training.csv'
X = pd.read_csv('../Data/' + file)
X['city'] = X['city'].astype('category')
X['gender'] = X['gender'].astype('category')
# X['bd'] = X['bd'].astype('category')
y = X['target']

X_num = X[numerical]

T = pd.read_csv('../Data/def_test.csv')
T['city'] = T['city'].astype('category')
T['gender'] = T['gender'].astype('category')
# T['bd'] = T['bd'].astype('category')

T_num = T[numerical]


Particiono

In [10]:
DummiesX = pd.get_dummies(data=X[categorical], prefix_sep='|')
T_cat = pd.get_dummies(data=T[categorical], prefix_sep='|')

print(T_cat.shape)
print(DummiesX.shape)

(X_num_train, X_num_test, X_num_val, y_train, y_test, y_val) = split(X_num, y, 0.5)
(X_cat_train, X_cat_test, X_cat_val, y_train, y_test, y_val) = split(DummiesX, y, 0.5)
del DummiesX , X_num
gc.collect()

(2556790, 43)
(7347633, 43)
Train 
Raw data: 
  Shape:  (7347633, 3) Type:  <class 'pandas.core.frame.DataFrame'>
Target 
Raw data: 
  Shape:  (7347633,)
Size of X:  (7347633, 3) 
Size of y:  (7347633,)

 New train shape:  (3673816, 3)  
 New test shape:  (1836908, 3) 
 New val shape:  (1836909, 3) 
 Type:  <class 'pandas.core.frame.DataFrame'>
Train 
Raw data: 
  Shape:  (7347633, 43) Type:  <class 'pandas.core.frame.DataFrame'>
Target 
Raw data: 
  Shape:  (7347633,)
Size of X:  (7347633, 43) 
Size of y:  (7347633,)

 New train shape:  (3673816, 43)  
 New test shape:  (1836908, 43) 
 New val shape:  (1836909, 43) 
 Type:  <class 'pandas.core.frame.DataFrame'>


432

In [11]:
X_cat_train[X_cat_train.columns] = X_cat_train[X_cat_train.columns].astype(np.uint8)
X_cat_test[X_cat_test.columns] = X_cat_test[X_cat_test.columns].astype(np.uint8)
X_cat_val[X_cat_val.columns] = X_cat_val[X_cat_val.columns].astype(np.uint8)
T_cat[T_cat.columns] = T_cat[T_cat.columns].astype(np.uint8)
X_cat_test.head()

,source_system_tab|discover,source_system_tab|explore,source_system_tab|listen with,source_system_tab|my library,source_system_tab|notification,source_system_tab|radio,source_system_tab|search,source_system_tab|settings,source_type|album,source_type|artist,...,city|13,city|14,city|15,city|16,city|17,city|18,city|19,city|20,city|21,city|22
5296846,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5530281,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5768063,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
174007,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3260021,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


Elimino las columnas que tienen cero para todas las filas de algun subconjunto

In [12]:
T_sum =  (T_cat.values.sum(axis=0) != 0)
X_train_sum = (X_cat_train.values.sum(axis=0) != 0)
X_test_sum = (X_cat_test.values.sum(axis=0) != 0)
X_val_sum = (X_cat_val.values.sum(axis=0) != 0)
index =np.logical_and(np.logical_and(X_train_sum, np.logical_and(X_test_sum, X_val_sum)),T_sum)
X_cat_train = X_cat_train.loc[:, index]
X_cat_test = X_cat_test.loc[:, index]
X_cat_val = X_cat_val.loc[:, index]
T_cat = T_cat.loc[:,index]
X_cat_test.head()

,source_system_tab|discover,source_system_tab|explore,source_system_tab|listen with,source_system_tab|my library,source_system_tab|notification,source_system_tab|radio,source_system_tab|search,source_system_tab|settings,source_type|album,source_type|artist,...,city|13,city|14,city|15,city|16,city|17,city|18,city|19,city|20,city|21,city|22
5296846,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5530281,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5768063,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
174007,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3260021,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [13]:
def Matrix_mult(*args):
	"""An internal method to multiply matrices."""
	return functools.reduce(np.dot, args)


# MCA

In [14]:
X_train = X_cat_train
size = [30000, X_train.shape[1]]
N_all = np.sum(X_train.values[0:size[0],:])

Z_residual = X_train.values[0:size[0],:] / N_all
Z_residual.shape

(30000, 43)

In [15]:
Sum_r = np.sum(Z_residual, axis=1)
Sum_c = np.sum(Z_residual, axis=0)
print(size, Sum_r.shape, Sum_c.shape, N_all)

[30000, 43] (30000,) (43,) 120000


In [16]:
Z_expected = np.outer(Sum_r, Sum_c)
Z_residual -= Z_expected
del Z_expected
gc.collect()

175

In [17]:
D_r_sqrt_mi = np.sqrt(np.diag(Sum_r ** -1))
del Sum_r
gc.collect()
D_c_sqrt_mi = np.sqrt(np.diag(Sum_c ** -1))
del Sum_c
gc.collect()

0

In [18]:
MCA_mat = Matrix_mult(D_r_sqrt_mi, Z_residual, D_c_sqrt_mi)

In [19]:
P, S, Q = np.linalg.svd(MCA_mat)

In [20]:
S_d = diagsvd(S, size[0], size[1])

In [21]:
F = Matrix_mult(D_r_sqrt_mi, P, S_d)  ## Column Space, contains linear combinations of columns
G = Matrix_mult(D_c_sqrt_mi, Q.T, S_d.T)  ## Row space, contains linear combinations of rows

In [22]:
del  Q, P, MCA_mat, S_d, D_r_sqrt_mi, D_c_sqrt_mi
gc.collect()

0

In [23]:
Lam = S ** 2
Expl_var = Lam / np.sum(Lam)

print('Eigen values are ', Lam)
print('Explained variance of eigen vectors are ', Expl_var)

data = {'Iλ': pd.Series(Lam),
		'τI': pd.Series(Expl_var)}
columns = ['Iλ', 'τI']#, 'Zλ', 'τZ', 'cλ', 'τc']
table2 = pd.DataFrame(data=data, columns=columns).fillna(0)
table2.index += 1
table2.loc['Σ'] = table2.sum()
table2.index.name = 'Factor'
np.round(table2.astype(float), 4)
table2.to_csv('table2.csv')

Eigen values are  [  4.93753543e-01   4.79178009e-01   4.72384164e-01   3.88979038e-01
   3.20453065e-01   2.69493420e-01   2.66237857e-01   2.63781779e-01
   2.57820088e-01   2.56809548e-01   2.55919412e-01   2.54637925e-01
   2.53079160e-01   2.50761735e-01   2.50266603e-01   2.50023848e-01
   2.50017248e-01   2.50006505e-01   2.50001041e-01   2.50000000e-01
   2.49888709e-01   2.49807984e-01   2.49684995e-01   2.49439793e-01
   2.49168718e-01   2.48646810e-01   2.46428003e-01   2.44725909e-01
   2.43978610e-01   2.42881079e-01   2.41809028e-01   2.37637207e-01
   2.34560351e-01   2.25745245e-01   1.79516254e-01   1.11265521e-01
   3.13673489e-02   2.27850430e-02   7.05940346e-03   7.68199526e-28
   3.59809370e-28   5.67781336e-29   2.57060330e-29]
Explained variance of eigen vectors are  [  5.06413890e-02   4.91464624e-02   4.84496578e-02   3.98952859e-02
   3.28669810e-02   2.76403508e-02   2.73064469e-02   2.70545414e-02
   2.64430859e-02   2.63394408e-02   2.62481448e-02   2.6116

In [30]:
columns_keep = 32
print('Varianza explicada tomando '+str(columns_keep)+' features: ', table2['τI'][0:columns_keep].sum())

Varianza explicada tomando 32 features:  0.916687265022


In [31]:
X_cat_trainMatrix= Matrix_mult(X_train.values, G[:, :columns_keep]) / S[:columns_keep] / 10
X_cat_testMatrix = Matrix_mult(X_cat_test.values, G[:, :columns_keep]) / S[:columns_keep] / 10
X_cat_valMatrix = Matrix_mult(X_cat_val.values, G[:, :columns_keep]) / S[:columns_keep] / 10
T_catMatrix = Matrix_mult(T_cat.values, G[:, :columns_keep]) / S[:columns_keep] / 10

In [32]:
names = ['V' + str(i) for i in range(columns_keep)]
X_cat_train = pd.DataFrame(X_cat_trainMatrix, columns=names, index = X_cat_train.index)
X_cat_test = pd.DataFrame(X_cat_testMatrix, columns=names, index =X_cat_test.index)
X_cat_val = pd.DataFrame(X_cat_valMatrix, columns=names, index =X_cat_val.index)  
T_cat = pd.DataFrame(T_catMatrix, columns=names, index =T_cat.index)



In [33]:
X_cat_train.head()


,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31
3584192,-0.046323,-0.035330,-0.337287,-0.181416,0.035752,-0.083739,0.231316,-0.271616,-0.110380,0.016966,...,0.117481,0.162748,0.148456,0.172998,-0.176801,-0.057461,0.208418,-0.023000,-0.001778,0.063576
2303057,-0.083554,1.648534,0.209081,-0.081982,-0.023442,0.244158,0.033611,-0.083684,-0.129399,0.288934,...,-0.037878,0.156126,0.236211,-0.133916,-0.102646,-0.109020,-0.034161,0.236110,-0.136040,-0.055688
3163611,-0.065856,0.009130,-0.187552,0.780613,0.069881,-0.306213,-0.152416,-0.152436,-0.218747,0.309962,...,-0.184683,0.095509,-0.044123,-0.286651,0.113336,0.043922,-0.215877,-0.371191,0.034738,0.241000
4019357,-0.063475,-0.017998,-0.341088,-0.196395,0.049089,-0.122805,0.008847,-0.212221,0.148161,0.053883,...,-0.057014,-0.163927,0.067718,0.022780,-0.018165,-0.024479,0.057480,-0.036147,0.167421,-0.167926
519966,-0.098290,-0.067233,0.299922,-0.057545,0.026488,-0.360577,0.138227,-0.072134,-0.107620,0.127257,...,-0.080382,-0.096567,0.244673,-0.057096,-0.039162,-0.105541,-0.097439,0.151130,-0.173748,-0.114430


In [34]:
T_cat.head()

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31
0,-0.080918,-0.104303,0.228060,-0.038174,0.006168,0.328695,-0.020573,-0.123637,0.000393,-0.006544,...,-0.030186,-0.067352,-0.042364,0.066318,-0.045732,0.028388,0.020373,-0.080974,-0.004696,-0.087426
1,-0.080918,-0.104303,0.228060,-0.038174,0.006168,0.328695,-0.020573,-0.123637,0.000393,-0.006544,...,-0.030186,-0.067352,-0.042364,0.066318,-0.045732,0.028388,0.020373,-0.080974,-0.004696,-0.087426
2,-0.059609,-0.027562,-0.414829,-0.246646,0.314587,0.110627,-0.041697,-0.035214,0.057341,-0.404981,...,-0.031255,0.004614,0.002772,0.160989,0.489152,0.124579,-0.191875,0.381694,-0.045781,0.172161
3,1.024993,-0.035827,0.112043,-0.088318,-0.022919,0.035451,-0.011118,-0.013672,-0.240868,0.036386,...,-0.594593,-0.428852,-0.514047,-0.967091,0.257502,-0.294126,0.801905,0.601493,0.054962,0.491663
4,1.024993,-0.035827,0.112043,-0.088318,-0.022919,0.035451,-0.011118,-0.013672,-0.240868,0.036386,...,-0.594593,-0.428852,-0.514047,-0.967091,0.257502,-0.294126,0.801905,0.601493,0.054962,0.491663


# Saving data

In [35]:
concat_Train = pd.concat([X_cat_train, X_num_train], axis=1, ignore_index=False)
concat_Test = pd.concat([X_cat_test, X_num_test], axis=1, ignore_index=False)
concat_Val = pd.concat([X_cat_val, X_num_val], axis=1, ignore_index=False)
concat_T= pd.concat([T_cat, T_num], axis=1, ignore_index=False)

In [36]:
concat_Train = pd.DataFrame(stdc().fit_transform(concat_Train.values), columns=concat_Train.columns, index = concat_Train.index)
concat_Test = pd.DataFrame(stdc().fit_transform(concat_Test.values), columns=concat_Test.columns, index = concat_Test.index)
concat_Val = pd.DataFrame(stdc().fit_transform(concat_Val.values), columns=concat_Val.columns, index = concat_Val.index)
concat_T = pd.DataFrame(stdc().fit_transform(concat_T.values), columns=concat_T.columns, index = concat_T.index)

In [37]:
concat_Train.isnull().sum().sum()

0

In [38]:
concat_Train.head()

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V25,V26,V27,V28,V29,V30,V31,song_length,song_year,bd
3584192,-0.166333,-0.125223,-1.234796,-0.728350,0.167874,-0.405933,1.092883,-1.336103,-0.573128,0.083900,...,0.857565,-0.884116,-0.298335,1.051222,-0.110716,-0.017881,0.326611,-0.393152,0.537425,0.739186
2303057,-0.298257,6.047501,0.755100,-0.330484,-0.106647,1.177150,0.156714,-0.407493,-0.672057,1.431586,...,-0.655747,-0.508179,-0.563703,-0.168925,1.187187,-0.719109,-0.281537,-0.032418,0.227031,0.549330
3163611,-0.235547,0.037759,-0.689452,3.121005,0.326157,-1.480039,-0.724161,-0.747212,-1.136810,1.535784,...,-1.408844,0.586771,0.223464,-1.082933,-1.854829,0.172834,1.231329,0.469210,-0.393756,0.359475
4019357,-0.227111,-0.061686,-1.248636,-0.788288,0.229727,-0.594544,0.039449,-1.042617,0.771689,0.266832,...,0.116878,-0.079889,-0.128581,0.292022,-0.176568,0.865810,-0.853861,-0.664094,0.692622,-1.096087
519966,-0.350475,-0.242171,1.085947,-0.232706,0.124911,-1.742511,0.652087,-0.350423,-0.558776,0.630425,...,-0.276969,-0.186338,-0.545797,-0.487203,0.761515,-0.916046,-0.581075,-0.382306,0.692622,0.675901


In [39]:
concat_Train['target'] = y_train
concat_Test['target']= y_test
concat_Val['target']= y_val


In [40]:
concat_T.to_csv('../Data/preprocessedOriginalTMCA.csv')
concat_Train.to_csv('../Data/preprocessedTrainMCA.csv')
concat_Test.to_csv('../Data/preprocessedTestMCA.csv')
concat_Val.to_csv('../Data/preprocessedValMCA.csv')